In [12]:
import pandas as pd
import glob
#Map the UCI Calendar excel output to the Continental circuits using the UCI's country names
continents = {'Americas': ['ANTIGUA AND BARBUDA', 'ARGENTINA', 'ARUBA', 'BELIZE', 'BERMUDA', 
                           'BOLIVARIAN REPUBLIC OF VENEZUELA', 'BOLIVIA', 'BRAZIL', 'CANADA',
                           'CHILE', 'COLOMBIA', 'CUBA', 'DOMINICAN REPUBLIC', 'ECUADOR', 
                           'EL SALVADOR', 'HONDURAS', 'PANAMA', 'PARAGUAY', 'PUERTO RICO', 
                           'SAINT VINCENT AND THE GRENADINES', 'TRINIDAD AND TOBAGO', 
                           'UNITED STATES OF AMERICA', 'URUGUAY','MEXICO', 'COSTA RICA','GUATEMALA'],
              'Africas' : ['ALBANIA', 'ALGERIA', 'ANGOLA', 'CAMEROON', 'CONGO', 'COTE D\'IVOIRE',
                           'EGYPT', 'ETHIOPIA', 'GUYANA', 'LIBYA', 'MALI', 'MAURITIUS', 'MOROCCO',
                           'NAMIBIA', 'RWANDA', 'SENEGAL', 'SOUTH AFRICA', 'SWAZILAND', 'TUNISIA', 
                           'UGANDA', 'ZIMBABWE', 'GABON','ERITREA', 'BURKINA FASO'],
              'Asia' : ['AZERBAIJAN', 'BAHRAIN', 'BRUNEI DARUSSALAM', 'CHINESE TAIPEI', 'GEORGIA',
                        'INDIA', 'ISLAMIC REPUBLIC OF IRAN', 'ISRAEL', 'KAZAKHSTAN', 'KOREA', 'KUWAIT', 
                        'KYRGYZSTAN', 'LEBANON', 'MALAYSIA', 'MONGOLIA', 'MYANMAR', 'OMAN', 'PHILIPPINES',
                        'RUSSIAN FEDERATION', 'SINGAPORE', 'SRI LANKA', 'SYRIAN ARAB REPUBLIC', 'THAILAND',
                        'TURKEY', 'UNITED ARAB EMIRATES', 'VIETNAM','HONG KONG, CHINA','QATAR', 'UZBEKISTAN',
                        'INDONESIA', 'JAPAN','PEOPLE\'S REPUBLIC OF CHINA'],
              'Europe' : ['AUSTRIA', 'BELARUS', 'BELGIUM', 'BOSNIA AND HERZEGOVINA', 'BULGARIA', 'CROATIA',
                          'CYPRUS', 'CZECH REPUBLIC', 'DENMARK', 'ESTONIA', 'FINLAND', 
                          'FORMER YUGOSLAV REPUBLIC OF MACEDONIA', 'FRANCE', 'GERMANY', 'GREAT BRITAIN', 
                          'GREECE', 'HUNGARY', 'ICELAND', 'IRELAND', 'ITALY', 'KOSOVO', 'LATVIA', 'LITHUANIA',
                          'LUXEMBOURG', 'MONTENEGRO', 'NETHERLANDS', 'NORWAY', 'POLAND', 'PORTUGAL', 
                          'REPUBLIC OF MOLDOVA', 'ROMANIA', 'SAN MARINO', 'SERBIA', 
                          'SLOVAKIA', 'SLOVENIA', 'SWEDEN', 'SWITZERLAND', 'UKRAINE','SPAIN'],
              'Oceania' : ['NEW ZEALAND','AUSTRALIA']}

In [13]:
#Pull in every excel sheet from the current folder that starts with Calendar and ends in xlsx. YMMV!

all_data = pd.DataFrame()
for f in glob.glob('Calendar*.xlsx'):
    df = pd.read_excel(f, header=1, encoding='UTF-8')
    all_data = all_data.append(df,ignore_index=True)


In [14]:
#Examine the head and length to make sure it's kosher
all_data.head()
len(all_data)

7985

In [15]:
#Get rid of unwanted columns.
del all_data['EMail']
del all_data['WebSite']
del all_data['Calendar']
del all_data['Venue']
all_data.head()

,Date From,Date To,Name,Country,Category,Class
0,29/10/2018,29/10/2018,Gran Premio ICODER,COSTA RICA,WE,1.2
1,30/10/2018,30/10/2018,Gran Premio Comite Olimpico Nacional Femenino,COSTA RICA,WE,1.2
2,23/10/2018,31/10/2018,Tour of Hainan,PEOPLE'S REPUBLIC OF CHINA,ME,2.HC
3,23/10/2018,01/11/2018,58 Vuelta a Guatemala,GUATEMALA,ME,2.2
4,26/10/2018,04/11/2018,Tour du Faso,BURKINA FASO,ME,2.2


In [16]:
#Extract the year from the start date noting that UCI 'seasons' differ from calendar years
#I will need to write code that changes seasons for Africa, Asia and or Oceania

all_data['Year'] = pd.DatetimeIndex(all_data['Date From']).year
#If you want you can just export the combined data
# writer = pd.ExcelWriter('combined.xlsx')
# all_data.to_excel(writer,'Sheet1')

# writer.save()

In [22]:
#We need to make the dictionary of lists one big dictionary to add the Continents

cont_dict_converted = {k: oldk for oldk, oldv in continents.items() for k in oldv}

#Note there are some events that cross borders or Continental championships that have no Continent.
#Add some code here to handle these cases.

all_data['Continent'] = all_data['Country'].map(cont_dict_converted).fillna('Stateless')

#Look at the numbers of races per continent
grouped_df = all_data.groupby('Year')
grouped_df.count()

,Date From,Date To,Name,Country,Category,Class,Continent
Year,,,,,,,
2005,22,22,22,22,22,22,22
2006,818,818,818,814,818,818,818
2007,749,749,749,749,749,749,749
2008,50,50,50,50,50,50,50
2009,554,554,554,554,554,554,554
2010,529,529,529,529,529,529,529
2011,525,525,525,525,523,525,525
2012,42,42,42,42,42,42,42
2013,662,662,662,662,662,662,662


In [8]:
conti_count = grouped_df["Name"].count()
conti_count.head()

Continent
Africas      348
Americas     851
Asia         840
Europe      5708
Oceania      221
Name: Name, dtype: int64

In [85]:
#add in the coordinates for each race so we can geolocate them on a map later

uci_country_coord = pd.DataFrame()
uci_country_coord = pd.read_excel('uci_country_coord.xlsx', encoding='UTF-8')
uci_df = pd.merge(all_data, uci_country_coord, right_on='uci_name', left_on='Country', how="left")

df_nulls = all_data[all_data.isnull().any(axis=1)]
df_nulls.head()

,Date From,Date To,Name,Country,Category,Class,Year,Continent
14,17/11/2018,18/11/2018,National Road Championships - Uzbekistan,UZBEKISTAN,NaN,CN,2018,Asia
18,21/11/2018,25/11/2018,Africa Cup,ERITREA,WE,NaN,2018,Africas
19,21/11/2018,25/11/2018,Africa Cup,ERITREA,ME,NaN,2018,Africas
20,07/12/2018,07/12/2018,National Road Championships - Qatar (IRR),QATAR,NaN,CN,2018,Asia
25,04/01/2019,06/01/2019,National Road Championships - New Zealand,NEW ZEALAND,NaN,CN,2019,Oceania


In [79]:
uci_country_coord.head(1)

,uci_name,continent,name,country,latitude,longitude
0,ALBANIA,AFRICA,Albania,AL,41.153332,20.168331
